# 可选择模型

In [8]:
# ==========================================
# 今日头条热榜 -> (Qwen / DeepSeek 可切换) -> 利多/利空展示
# 稳定版：
#  - 强制 JSON 输出（response_format=json_object）
#  - 标题/文本净化（避免 ASCII 双引号破坏 JSON）
#  - 本地严格校验 + 重试（区分：429/5xx/空content/截断/解析失败）
#  - 截断自适应：先加 max_tokens，再减输出条数
#  - 可选弹窗（浏览器打开 HTML）
#  - 每次运行保存 Excel（同文件两张 sheet：利多/利空；文件名=时间戳避免重复）
# ==========================================

# =========================
# A) 参数区（你主要改这里）
# =========================
CONFIG = {
    # ✅ 选择模型提供商： "deepseek" 或 "qwen"
    "PROVIDER": "deepseek",

    # 定时运行
    "ENABLE_SCHEDULER": False,
    "INTERVAL_MINUTES": 30,

    # 热搜抓取
    "TOP_K": 30,
    "TOUTIAO_TIMEOUT": 12,

    # Qwen（DashScope OpenAI 兼容接口）
    "QWEN_API_KEY": "sk-b8a5d51908184663854d1b5c8e73e25e",
    "QWEN_MODEL": "qwen-plus",
    "QWEN_BASE_URL": "https://dashscope.aliyuncs.com/compatible-mode/v1",
    "QWEN_TIMEOUT": 60,

    # DeepSeek（OpenAI 兼容接口）
    "DEEPSEEK_API_KEY": "sk-1d213aa830694d2ba292a75d06cd5d77",
    "DEEPSEEK_MODEL": "deepseek-chat",
    "DEEPSEEK_BASE_URL": "https://api.deepseek.com/v1",
    "DEEPSEEK_TIMEOUT": 60,

    # 生成参数：从较小开始；检测截断会自动抬升到 MAX_TOKENS_MAX
    "TEMPERATURE": 0.0,
    "MAX_TOKENS_START": 1400,
    "MAX_TOKENS_MAX": 4200,      # 太大可能被服务端拒绝，会自动回退

    # 输出控制：若多次截断会自动降低条数
    "MAX_ITEMS_EACH_SIDE": 12,
    "MAX_IGNORED": 12,
    "MIN_ITEMS_EACH_SIDE": 5,
    "MIN_IGNORED": 0,

    # 重试控制
    "MAX_RETRIES": 3,
    "BASE_RETRY_SLEEP_SECONDS": 0.6,
    "MAX_BACKOFF_SECONDS": 20,

    # 调试
    "DEBUG_SHOW_RAW_ON_FAIL": True,
    "DEBUG_SHOW_API_ERROR_TEXT": False,

    # 结果展示：弹出窗口
    "ENABLE_POPUP": True,
    "POPUP_TITLE_PREFIX": "热搜利多/利空分析",

    # 结果保存：Excel
    "ENABLE_SAVE_EXCEL": True,
    "OUTPUT_DIR": "./hotsearch_stock_outputs",
}

# =========================
# B) 依赖
# =========================
import os
import time
import json
import re
import random
import webbrowser
from datetime import datetime
from typing import List, Dict, Any, Tuple, Optional

import requests
import pandas as pd

try:
    from IPython.display import display  # type: ignore
except Exception:
    display = None

# DataFrame 显示：全部行列 + 完整字符串
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 0)
pd.set_option("display.max_colwidth", None)
pd.set_option("display.expand_frame_repr", False)


# =========================
# 0) 小工具：标题净化（避免 ASCII 双引号破坏 JSON）
# =========================
def sanitize_title_for_model(title: str) -> str:
    """
    把标题里的 ASCII 双引号替换成中文引号，避免模型原样拷贝导致 JSON 字符串未转义。
    同时去掉控制字符，避免字符串内出现非法换行/控制符。
    """
    if title is None:
        return ""
    s = str(title)
    s = s.replace('"', "“")
    s = re.sub(r"[\x00-\x08\x0b\x0c\x0e-\x1f]", " ", s)
    s = s.replace("\r", " ").replace("\n", " ")
    s = re.sub(r"\s+", " ", s).strip()
    return s


# =========================
# 1) 抓取今日头条热榜
# =========================
def fetch_toutiao_hot(top_k: int = 30, timeout: int = 12) -> List[Dict[str, Any]]:
    url = "https://www.toutiao.com/hot-event/hot-board/?origin=toutiao_pc"
    headers = {
        "User-Agent": (
            "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
            "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0 Safari/537.36"
        ),
        "Referer": "https://www.toutiao.com/",
        "Accept": "application/json,text/plain,*/*",
    }
    r = requests.get(url, headers=headers, timeout=timeout)
    r.raise_for_status()
    payload = r.json()

    data = payload.get("data")
    if not isinstance(data, list):
        raise ValueError(f"Unexpected response schema. Top keys: {list(payload.keys())}")

    items: List[Dict[str, Any]] = []
    for i, it in enumerate(data[:top_k], start=1):
        title = it.get("Title") or it.get("title") or ""
        hot_value = it.get("HotValue") or it.get("hot_value") or 0
        link = it.get("Url") or it.get("url") or ""

        try:
            hot_value_num = float(hot_value)
        except Exception:
            hot_value_num = None

        items.append({
            "rank": i,
            "title": str(title).strip(),
            "hot_value": hot_value_num,
            "url": str(link).strip(),
        })

    return [x for x in items if x["title"]]


# =========================
# 2) 通用：JSON 清理/解析/校验（重要：不把中文引号替换成 ASCII 引号）
# =========================
def clean_text(s: str) -> str:
    s = (s or "").strip()
    s = re.sub(r"^```(?:json)?\s*", "", s, flags=re.IGNORECASE)
    s = re.sub(r"\s*```$", "", s)

    s = re.sub(r"\bNaN\b", "null", s)
    s = re.sub(r"\bInfinity\b", "null", s)
    s = re.sub(r"\b-Infinity\b", "null", s)

    s = re.sub(r",\s*([}\]])", r"\1", s)
    return s


def strict_parse_json(s: str) -> Dict[str, Any]:
    s2 = clean_text(s).strip()
    if not (s2.startswith("{") and s2.endswith("}")):
        raise ValueError("Output is not a pure JSON object (missing { } boundaries).")
    obj = json.loads(s2)
    if not isinstance(obj, dict):
        raise ValueError("JSON is not an object.")
    return obj


def validate_result(obj: Dict[str, Any]) -> None:
    for k in ["bullish", "bearish", "ignored"]:
        if k not in obj:
            raise ValueError(f"Missing key: {k}")
        if not isinstance(obj[k], list):
            raise ValueError(f"Key '{k}' must be a list.")

    def _check_side(side_name: str):
        for it in obj.get(side_name, []):
            if not isinstance(it, dict):
                raise ValueError(f"{side_name} item not object.")
            for need in ["rank", "topic", "industries", "mechanism", "confidence"]:
                if need not in it:
                    raise ValueError(f"{side_name} missing field: {need}")
            if not isinstance(it["industries"], list):
                raise ValueError(f"{side_name}.industries must be list")
            c = it["confidence"]
            if not (isinstance(c, (int, float)) and 0.0 <= float(c) <= 1.0):
                raise ValueError(f"{side_name}.confidence out of range: {c}")

    _check_side("bullish")
    _check_side("bearish")


# =========================
# 3) Prompt（短 + 强约束：宁可少条也要完整 JSON）
# =========================
def build_messages(items: List[Dict[str, Any]], max_each_side: int, max_ignored: int) -> List[Dict[str, str]]:
    hot_lines = []
    for it in items:
        hv = it["hot_value"]
        hv_str = f"{hv:.0f}" if isinstance(hv, (int, float)) and hv is not None else ""
        safe_title = sanitize_title_for_model(it["title"])
        hot_lines.append(f'{it["rank"]}. {safe_title} (HotValue={hv_str})')

    system = (
        "你是严谨的中文金融信息分析助手。"
        "你必须只输出json对象（严格JSON），不要任何解释文字。"
        "如果输出可能过长，宁可减少条目数，也必须保证输出为完整合法JSON。"
    )

    user = f"""
请基于下面的“今日头条热榜标题列表”，挑选出可能引起相关行业/板块短期股价波动的标题，并分为：
- bullish（利多）
- bearish（利空）
- ignored（不太相关）

严格要求（必须遵守）：
- 只输出严格json（JSON对象），不要 markdown，不要解释，不要任何前后缀。
- 输出必须以 {{ 开头，以 }} 结尾。
- 每组 bullish/bearish 最多 {max_each_side} 条；ignored 最多 {max_ignored} 条。
- mechanism 必须非常短：<= 25 个汉字（不要换行）。
- 不要编造具体股票代码；只给行业/板块。
- 如果标题含有引号，请使用中文引号（“ ”）或单引号，不要使用ASCII双引号字符 " 。

固定 JSON Schema（字段名必须一致）：
{{
  "bullish": [{{"rank": 1, "topic": "...", "industries": ["..."], "mechanism": "...", "confidence": 0.0}}],
  "bearish":  [{{"rank": 1, "topic": "...", "industries": ["..."], "mechanism": "...", "confidence": 0.0}}],
  "ignored":  [{{"rank": 1, "topic": "...", "reason": "..."}}]
}}

热榜标题列表：
{chr(10).join(hot_lines)}
""".strip()

    return [
        {"role": "system", "content": system},
        {"role": "user", "content": user},
    ]


# =========================
# 4) Provider 调用（DeepSeek / Qwen）
# =========================
class TemporaryAPIError(Exception):
    def __init__(self, msg: str, status_code: Optional[int] = None, retry_after: Optional[float] = None):
        super().__init__(msg)
        self.status_code = status_code
        self.retry_after = retry_after


def _post_chat_completions(
    base_url: str,
    api_key: str,
    timeout: int,
    body: Dict[str, Any],
) -> Dict[str, Any]:
    endpoint = base_url.rstrip("/") + "/chat/completions"
    headers = {"Authorization": f"Bearer {api_key}", "Content-Type": "application/json"}

    try:
        r = requests.post(endpoint, headers=headers, json=body, timeout=timeout)
    except requests.RequestException as e:
        raise TemporaryAPIError(f"Network error: {e}", status_code=None) from e

    # 临时错误：限流/服务端异常
    if r.status_code in (429, 500, 502, 503, 504):
        retry_after = None
        ra = r.headers.get("Retry-After")
        if ra:
            try:
                retry_after = float(ra)
            except Exception:
                retry_after = None
        msg = f"Temporary HTTP {r.status_code}"
        if CONFIG.get("DEBUG_SHOW_API_ERROR_TEXT"):
            msg += f": {r.text[:1000]}"
        raise TemporaryAPIError(msg, status_code=r.status_code, retry_after=retry_after)

    if not r.ok:
        raise RuntimeError(f"HTTP {r.status_code}: {r.text[:1500]}")

    return r.json()


def call_provider_chat(
    provider: str,
    messages: List[Dict[str, str]],
    temperature: float,
    max_tokens: int,
) -> Tuple[str, Optional[str], Dict[str, Any]]:
    """
    返回 (content_text, finish_reason, raw_response)
    """
    provider = provider.lower().strip()
    if provider not in ("deepseek", "qwen"):
        raise ValueError("CONFIG['PROVIDER'] must be 'deepseek' or 'qwen'.")

    if provider == "deepseek":
        base_url = CONFIG["DEEPSEEK_BASE_URL"]
        api_key = CONFIG["DEEPSEEK_API_KEY"]
        model = CONFIG["DEEPSEEK_MODEL"]
        timeout = CONFIG["DEEPSEEK_TIMEOUT"]
    else:
        base_url = CONFIG["QWEN_BASE_URL"]
        api_key = CONFIG["QWEN_API_KEY"]
        model = CONFIG["QWEN_MODEL"]
        timeout = CONFIG["QWEN_TIMEOUT"]

    body: Dict[str, Any] = {
        "model": model,
        "messages": messages,
        "temperature": temperature,
        "max_tokens": max_tokens,
        "stream": False,
        "response_format": {"type": "json_object"},  # 两家都尽量走 JSON mode
    }

    raw = _post_chat_completions(base_url, api_key, timeout, body)

    choice0 = raw["choices"][0]
    finish_reason = choice0.get("finish_reason")
    msg = choice0["message"]
    content = msg.get("content")

    # 某些实现可能 tool_calls/arguments（可选）
    tool_calls = msg.get("tool_calls")
    if isinstance(tool_calls, list) and tool_calls:
        fn = tool_calls[0].get("function", {})
        args = fn.get("arguments")
        if isinstance(args, str) and args.strip():
            return args, finish_reason, raw

    if not isinstance(content, str) or not content.strip():
        raise TemporaryAPIError("Empty content returned (JSON mode).", status_code=200)

    return content, finish_reason, raw


# =========================
# 5) 重试策略（自适应：截断->加tokens/减条目）
# =========================
def _sleep_with_backoff(attempt: int, base: float, cap: float) -> None:
    wait = min(cap, base * (1.7 ** (attempt - 1)))
    wait *= (0.85 + 0.3 * random.random())
    time.sleep(wait)


def analyze_with_retries(items: List[Dict[str, Any]]) -> Dict[str, Any]:
    last_text: Optional[str] = None
    last_err: Optional[Exception] = None
    last_finish: Optional[str] = None

    max_tokens = int(CONFIG["MAX_TOKENS_START"])
    max_each_side = int(CONFIG["MAX_ITEMS_EACH_SIDE"])
    max_ignored = int(CONFIG["MAX_IGNORED"])

    for attempt in range(1, CONFIG["MAX_RETRIES"] + 1):
        messages = build_messages(items, max_each_side=max_each_side, max_ignored=max_ignored)

        try:
            text, finish_reason, raw = call_provider_chat(
                provider=CONFIG["PROVIDER"],
                messages=messages,
                temperature=CONFIG["TEMPERATURE"],
                max_tokens=max_tokens,
            )
            last_text, last_finish = text, finish_reason

            if finish_reason == "length":
                raise ValueError("finish_reason=length (likely truncated JSON)")

            obj = strict_parse_json(text)
            validate_result(obj)
            return obj

        except TemporaryAPIError as e:
            last_err = e
            # Retry-After 优先
            if e.retry_after is not None and e.retry_after > 0:
                time.sleep(min(CONFIG["MAX_BACKOFF_SECONDS"], e.retry_after))
            else:
                _sleep_with_backoff(attempt, base=1.0, cap=CONFIG["MAX_BACKOFF_SECONDS"])

        except Exception as e:
            last_err = e

            # 解析失败/疑似截断：加 tokens；到顶后减输出条数
            likely_trunc = (
                "finish_reason=length" in str(e)
                or (last_text is not None and not clean_text(last_text).strip().endswith("}"))
            )

            if likely_trunc:
                if max_tokens < CONFIG["MAX_TOKENS_MAX"]:
                    max_tokens = min(CONFIG["MAX_TOKENS_MAX"], int(max_tokens * 1.4) + 200)
                else:
                    if max_each_side > CONFIG["MIN_ITEMS_EACH_SIDE"]:
                        max_each_side = max(CONFIG["MIN_ITEMS_EACH_SIDE"], max_each_side - 2)
                    if max_ignored > CONFIG["MIN_IGNORED"]:
                        max_ignored = max(CONFIG["MIN_IGNORED"], max_ignored - 3)

            time.sleep(CONFIG["BASE_RETRY_SLEEP_SECONDS"])

    if CONFIG.get("DEBUG_SHOW_RAW_ON_FAIL") and last_text:
        print("\n[DEBUG] Provider:", CONFIG["PROVIDER"])
        print("[DEBUG] Last finish_reason:", last_finish)
        print("[DEBUG] Last model output (first 1200 chars):")
        print(clean_text(last_text)[:1200])

    raise RuntimeError(
        f"Failed to obtain valid JSON after {CONFIG['MAX_RETRIES']} attempts. Last error: {last_err}"
    )


# =========================
# 6) DataFrame + 展示
# =========================
def to_df(lst: List[Dict[str, Any]]) -> pd.DataFrame:
    if not lst:
        return pd.DataFrame()
    df = pd.DataFrame(lst).copy()
    if "industries" in df.columns:
        df["industries"] = df["industries"].apply(lambda x: "，".join(x) if isinstance(x, list) else x)
    return df


def show_df(df: pd.DataFrame):
    if display is not None:
        display(df)
    else:
        print(df.to_string(index=False))


# =========================
# 7) 弹出窗口展示（浏览器打开本地 HTML）
# =========================
def popup_show_dfs(bullish_df: pd.DataFrame, bearish_df: pd.DataFrame, title: str, output_dir: str):
    os.makedirs(output_dir, exist_ok=True)

    def _df_to_html(df: pd.DataFrame) -> str:
        if df is None or df.empty:
            return "<p><em>（无数据）</em></p>"
        return df.to_html(index=False, escape=True)

    html = f"""
<!doctype html>
<html lang="zh">
<head>
<meta charset="utf-8">
<title>{title}</title>
<style>
body {{ font-family: -apple-system,BlinkMacSystemFont,"Segoe UI",Roboto,Arial; margin: 20px; }}
h1 {{ margin-bottom: 12px; }}
h2 {{ margin-top: 22px; }}
table {{ border-collapse: collapse; width: 100%; }}
th, td {{ border: 1px solid #ddd; padding: 8px; vertical-align: top; }}
th {{ background: #f6f6f6; }}
tr:nth-child(even) {{ background: #fafafa; }}
.small {{ color: #666; font-size: 12px; }}
</style>
</head>
<body>
<h1>{title}</h1>
<div class="small">生成时间：{datetime.now().strftime("%Y-%m-%d %H:%M:%S")}</div>

<h2>📈 可能利多（正面催化）</h2>
{_df_to_html(bullish_df)}

<h2>📉 可能利空（负面冲击）</h2>
{_df_to_html(bearish_df)}

</body>
</html>
""".strip()

    ts = datetime.now().strftime("%Y%m%d_%H%M%S")
    filepath = os.path.abspath(os.path.join(output_dir, f"popup_{ts}.html"))
    with open(filepath, "w", encoding="utf-8") as f:
        f.write(html)

    webbrowser.open(f"file://{filepath}", new=1)


# =========================
# 8) 保存 Excel（同一文件两张 sheet）
# =========================
def save_dfs_to_excel(bullish_df: pd.DataFrame, bearish_df: pd.DataFrame, ts_str: str, output_dir: str) -> str:
    os.makedirs(output_dir, exist_ok=True)

    base_name = f"hotsearch_{ts_str}.xlsx"
    path = os.path.join(output_dir, base_name)

    if os.path.exists(path):
        i = 1
        while True:
            alt = os.path.join(output_dir, f"hotsearch_{ts_str}_{i}.xlsx")
            if not os.path.exists(alt):
                path = alt
                break
            i += 1

    with pd.ExcelWriter(path, engine="openpyxl") as writer:
        (bullish_df if bullish_df is not None else pd.DataFrame()).to_excel(writer, sheet_name="利多", index=False)
        (bearish_df if bearish_df is not None else pd.DataFrame()).to_excel(writer, sheet_name="利空", index=False)

    return os.path.abspath(path)


# =========================
# 9) 运行一次
# =========================
def run_once() -> Dict[str, Any]:
    items = fetch_toutiao_hot(CONFIG["TOP_K"], CONFIG["TOUTIAO_TIMEOUT"])
    result = analyze_with_retries(items)

    ts_display = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    ts_fname = datetime.now().strftime("%Y%m%d_%H%M%S")

    bullish_df = to_df(result.get("bullish", []))
    bearish_df = to_df(result.get("bearish", []))

    print(f"\n===== Run @ {ts_display} =====")
    print(f"Provider={CONFIG['PROVIDER']} | Fetched {len(items)} hot topics. "
          f"Model returned {len(result.get('bullish', []))} bullish / {len(result.get('bearish', []))} bearish.\n")

    print("📈 可能利多（正面催化）")
    show_df(bullish_df)

    print("\n📉 可能利空（负面冲击）")
    show_df(bearish_df)

    if CONFIG.get("ENABLE_SAVE_EXCEL"):
        saved_path = save_dfs_to_excel(
            bullish_df=bullish_df,
            bearish_df=bearish_df,
            ts_str=ts_fname,
            output_dir=CONFIG["OUTPUT_DIR"],
        )
        print(f"\n💾 已保存 Excel：{saved_path}")

    if CONFIG.get("ENABLE_POPUP"):
        title = f"{CONFIG.get('POPUP_TITLE_PREFIX', '热搜分析')} - {ts_display} ({CONFIG['PROVIDER']})"
        popup_show_dfs(bullish_df, bearish_df, title=title, output_dir=CONFIG["OUTPUT_DIR"])

    return result


# =========================
# 10) 定时循环（可选）
# =========================
def run_scheduler():
    interval_sec = int(CONFIG["INTERVAL_MINUTES"] * 60)
    print(f"Scheduler started. Provider={CONFIG['PROVIDER']}. Interval={CONFIG['INTERVAL_MINUTES']} minutes. Stop with Kernel Interrupt (⛔).")
    while True:
        try:
            run_once()
        except Exception as e:
            print(f"[ERROR] {e}")
        time.sleep(interval_sec)


# =========================
# 11) 执行入口
# =========================
_ = run_once()

if CONFIG["ENABLE_SCHEDULER"]:
    run_scheduler()



===== Run @ 2025-12-29 19:23:39 =====
Provider=deepseek | Fetched 30 hot topics. Model returned 7 bullish / 4 bearish.

📈 可能利多（正面催化）


,rank,topic,industries,mechanism,confidence
0,21,A股创年内连涨纪录 是新起点吗,A股市场,市场情绪提振，可能推动短期上涨。,0.70
1,18,数字人民币迎来重大调整,金融科技，数字货币,政策利好，促进相关技术应用和推广。,0.80
2,22,2026年起数字人民币余额按存款计息,金融科技，数字货币,增加吸引力，利好数字货币生态发展。,0.75
3,28,明年起我国调整部分商品关税,进出口贸易,关税调整可能降低进口成本，刺激消费。,0.60
4,3,2025“三农”成绩单,农业,政策支持或业绩提升，利好农业板块。,0.65
5,24,投资人蹲守大疆楼下抢人,科技，无人机,人才竞争激烈，反映行业高增长潜力。,0.70
6,9,大学生第一份工作应该选央国企吗,央国企,关注度提升，可能增强投资者信心。,0.50



📉 可能利空（负面冲击）


,rank,topic,industries,mechanism,confidence
0,2,中方要求彻查中巴公园拆除事件,中巴关系相关行业,外交紧张可能影响双边贸易和投资。,0.60
1,11,餐饮商户感染艾滋病？辽宁喀左辟谣,餐饮业,负面谣言可能短期冲击消费者信心。,0.55
2,16,俄为何将在俄日敏感地点军演两个月,地缘政治敏感行业,地缘紧张加剧，增加市场不确定性。,0.65
3,17,台军坦克上街不久就起火冒白烟,军工,事件可能引发对军工产品质量担忧。,0.50



💾 已保存 Excel：/Users/pz/VScode/Stock_Ana/hotsearch_stock_outputs/hotsearch_20251229_192339.xlsx


# 只使用deepseek的版本

In [5]:
# ==========================================
# 今日头条热榜 -> DeepSeek(JSON mode)分析 -> 利多/利空展示
# 稳定版（解决：截断/中文引号被误替换/偶发空content/真正重试）
# 新增/保留：
#  1) JSON-only + 本地严格校验 + 自动重试（最多N次）
#  2) 可选弹出窗口（浏览器打开本地 HTML 展示两个 DF）
#  3) 每次运行保存 Excel（同一文件两张 sheet：利多/利空；文件名=时间戳避免重复）
#  4) DataFrame 显示完整内容（全部行列+完整字符串）
# ==========================================

# =========================
# A) 参数区（你主要改这里）
# =========================
CONFIG = {
    # 定时运行
    "ENABLE_SCHEDULER": False,
    "INTERVAL_MINUTES": 30,

    # 热搜抓取
    "TOP_K": 30,
    "TOUTIAO_TIMEOUT": 12,

    # DeepSeek（OpenAI 兼容）
    "DEEPSEEK_API_KEY": "sk-1d213aa830694d2ba292a75d06cd5d77",
    "DEEPSEEK_MODEL": "deepseek-chat",
    "DEEPSEEK_BASE_URL": "https://api.deepseek.com/v1",
    "DEEPSEEK_TIMEOUT": 60,

    # 生成参数：从较小开始，若检测到截断会自动抬升到 MAX_TOKENS_MAX
    "TEMPERATURE": 0.0,
    "MAX_TOKENS_START": 1400,
    "MAX_TOKENS_MAX": 4200,  # 过大可能被服务端拒绝；会自动回退

    # 输出控制（若多次截断，会自动降低条数）
    "MAX_ITEMS_EACH_SIDE": 12,
    "MAX_IGNORED": 12,
    "MIN_ITEMS_EACH_SIDE": 5,
    "MIN_IGNORED": 0,

    # 重试控制
    "MAX_DEEPSEEK_RETRIES": 1,
    "BASE_RETRY_SLEEP_SECONDS": 0.6,
    "MAX_BACKOFF_SECONDS": 20,

    # 调试
    "DEBUG_SHOW_RAW_ON_FAIL": True,
    "DEBUG_SHOW_API_ERROR_TEXT": False,

    # 结果展示：弹出窗口
    "ENABLE_POPUP": True,
    "POPUP_TITLE_PREFIX": "热搜利多/利空分析",

    # 结果保存：Excel
    "ENABLE_SAVE_EXCEL": True,
    "OUTPUT_DIR": "./hotsearch_stock_outputs",
}

# =========================
# B) 依赖
# =========================
import os
import time
import json
import re
import random
import webbrowser
from datetime import datetime
from typing import List, Dict, Any, Tuple, Optional

import requests
import pandas as pd

try:
    from IPython.display import display  # type: ignore
except Exception:
    display = None

# DataFrame 显示：全部行列 + 完整字符串
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 0)
pd.set_option("display.max_colwidth", None)
pd.set_option("display.expand_frame_repr", False)


# =========================
# 0) 小工具：标题净化（避免 ASCII 双引号破坏 JSON）
# =========================
def sanitize_title_for_model(title: str) -> str:
    """
    把标题里的 ASCII 双引号替换成中文引号，避免模型原样拷贝导致 JSON 字符串未转义。
    同时去掉控制字符（\r \t 等），避免出现 JSON 字符串内非法换行。
    """
    if title is None:
        return ""
    s = str(title)

    # ASCII " -> 中文引号（成对替换很复杂，这里简单替换成“）
    s = s.replace('"', "“")

    # 去除控制字符（保留常规可读字符）
    s = re.sub(r"[\x00-\x08\x0b\x0c\x0e-\x1f]", " ", s)
    s = s.replace("\r", " ").replace("\n", " ")

    # 压缩多余空格
    s = re.sub(r"\s+", " ", s).strip()
    return s


# =========================
# 1) 抓取今日头条热榜
# =========================
def fetch_toutiao_hot(top_k: int = 30, timeout: int = 12) -> List[Dict[str, Any]]:
    url = "https://www.toutiao.com/hot-event/hot-board/?origin=toutiao_pc"
    headers = {
        "User-Agent": (
            "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
            "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0 Safari/537.36"
        ),
        "Referer": "https://www.toutiao.com/",
        "Accept": "application/json,text/plain,*/*",
    }
    r = requests.get(url, headers=headers, timeout=timeout)
    r.raise_for_status()
    payload = r.json()

    data = payload.get("data")
    if not isinstance(data, list):
        raise ValueError(f"Unexpected response schema. Top keys: {list(payload.keys())}")

    items: List[Dict[str, Any]] = []
    for i, it in enumerate(data[:top_k], start=1):
        title = it.get("Title") or it.get("title") or ""
        hot_value = it.get("HotValue") or it.get("hot_value") or 0
        link = it.get("Url") or it.get("url") or ""

        try:
            hot_value_num = float(hot_value)
        except Exception:
            hot_value_num = None

        items.append({
            "rank": i,
            "title": str(title).strip(),
            "hot_value": hot_value_num,
            "url": str(link).strip(),
        })

    return [x for x in items if x["title"]]


# =========================
# 2) DeepSeek 调用（OpenAI 兼容 chat/completions）
# =========================
class TemporaryAPIError(Exception):
    def __init__(self, msg: str, status_code: Optional[int] = None, retry_after: Optional[float] = None):
        super().__init__(msg)
        self.status_code = status_code
        self.retry_after = retry_after


def deepseek_chat_raw(
    api_key: str,
    base_url: str,
    model: str,
    messages: List[Dict[str, str]],
    timeout: int,
    temperature: float,
    max_tokens: int,
) -> Tuple[str, Dict[str, Any], Optional[str]]:
    """
    返回 (content_text, raw_json, finish_reason)
    """
    endpoint = base_url.rstrip("/") + "/chat/completions"
    headers = {"Authorization": f"Bearer {api_key}", "Content-Type": "application/json"}

    body: Dict[str, Any] = {
        "model": model,
        "messages": messages,
        "temperature": temperature,
        "max_tokens": max_tokens,
        "stream": False,
        "response_format": {"type": "json_object"},  # JSON mode
    }

    try:
        r = requests.post(endpoint, headers=headers, json=body, timeout=timeout)
    except requests.RequestException as e:
        raise TemporaryAPIError(f"Network error: {e}", status_code=None) from e

    if r.status_code in (429, 500, 502, 503, 504):
        retry_after = None
        ra = r.headers.get("Retry-After")
        if ra:
            try:
                retry_after = float(ra)
            except Exception:
                retry_after = None

        msg = f"Temporary HTTP {r.status_code}"
        if CONFIG.get("DEBUG_SHOW_API_ERROR_TEXT"):
            msg += f": {r.text[:1000]}"
        raise TemporaryAPIError(msg, status_code=r.status_code, retry_after=retry_after)

    if not r.ok:
        # 非临时错误：直接抛出
        raise RuntimeError(f"HTTP {r.status_code}: {r.text[:1500]}")

    raw = r.json()
    choice0 = raw["choices"][0]
    finish_reason = choice0.get("finish_reason")

    msg = choice0["message"]
    content = msg.get("content")

    # JSON mode 偶发空 content：当作可重试
    if not isinstance(content, str) or not content.strip():
        raise TemporaryAPIError("Empty content returned (JSON mode).", status_code=200)

    return content, raw, finish_reason


# =========================
# 3) JSON 清理/解析/校验（重要：不再把中文引号替换成 ASCII 引号）
# =========================
def clean_text(s: str) -> str:
    s = s.strip()
    # 去掉 ```json``` 包裹（保险）
    s = re.sub(r"^```(?:json)?\s*", "", s, flags=re.IGNORECASE)
    s = re.sub(r"\s*```$", "", s)

    # JSON 不允许 NaN/Infinity
    s = re.sub(r"\bNaN\b", "null", s)
    s = re.sub(r"\bInfinity\b", "null", s)
    s = re.sub(r"\b-Infinity\b", "null", s)

    # 尾逗号（保险）
    s = re.sub(r",\s*([}\]])", r"\1", s)
    return s


def strict_parse_json(s: str) -> Dict[str, Any]:
    s2 = clean_text(s).strip()
    if not (s2.startswith("{") and s2.endswith("}")):
        raise ValueError("Output is not a pure JSON object (missing { } boundaries).")
    obj = json.loads(s2)
    if not isinstance(obj, dict):
        raise ValueError("JSON is not an object.")
    return obj


def validate_result(obj: Dict[str, Any]) -> None:
    for k in ["bullish", "bearish", "ignored"]:
        if k not in obj:
            raise ValueError(f"Missing key: {k}")
        if not isinstance(obj[k], list):
            raise ValueError(f"Key '{k}' must be a list.")

    # 进一步校验字段/范围（尽量早发现“看似 JSON 但结构乱了”）
    def _check_side(side_name: str):
        for it in obj.get(side_name, []):
            if not isinstance(it, dict):
                raise ValueError(f"{side_name} item not object.")
            for need in ["rank", "topic", "industries", "mechanism", "confidence"]:
                if need not in it:
                    raise ValueError(f"{side_name} missing field: {need}")
            if not isinstance(it["industries"], list):
                raise ValueError(f"{side_name}.industries must be list")
            c = it["confidence"]
            if not (isinstance(c, (int, float)) and 0.0 <= float(c) <= 1.0):
                raise ValueError(f"{side_name}.confidence out of range: {c}")

    _check_side("bullish")
    _check_side("bearish")


# =========================
# 4) Prompt（更短、更不容易截断；强制“宁可少条目也要完整 JSON”）
# =========================
def build_messages(items: List[Dict[str, Any]], max_each_side: int, max_ignored: int) -> List[Dict[str, str]]:
    hot_lines = []
    for it in items:
        hv = it["hot_value"]
        hv_str = f"{hv:.0f}" if isinstance(hv, (int, float)) and hv is not None else ""
        safe_title = sanitize_title_for_model(it["title"])
        hot_lines.append(f'{it["rank"]}. {safe_title} (HotValue={hv_str})')

    system = (
        "你是严谨的中文金融信息分析助手。"
        "你必须只输出json对象（严格JSON），不要任何解释文字。"
        "如果你发现输出可能过长，宁可减少条目数，也必须保证输出为完整合法JSON。"
    )

    user = f"""
请基于下面的“今日头条热榜标题列表”，挑选出可能引起相关行业/板块短期股价波动的标题，并分为：
- bullish（利多）
- bearish（利空）
- ignored（不太相关）

严格要求（必须遵守）：
- 只输出严格json（JSON对象），不要 markdown，不要解释，不要任何前后缀。
- 输出必须以 {{ 开头，以 }} 结尾。
- 每组 bullish/bearish 最多 {max_each_side} 条；ignored 最多 {max_ignored} 条。
- mechanism 必须非常短：<= 25 个汉字（不要换行）。
- 不要编造具体股票代码；只给行业/板块。
- 如果标题含有引号，请使用中文引号（“ ”）或单引号，不要使用ASCII双引号字符 " 。

固定 JSON Schema（字段名必须一致）：
{{
  "bullish": [{{"rank": 1, "topic": "...", "industries": ["..."], "mechanism": "...", "confidence": 0.0}}],
  "bearish":  [{{"rank": 1, "topic": "...", "industries": ["..."], "mechanism": "...", "confidence": 0.0}}],
  "ignored":  [{{"rank": 1, "topic": "...", "reason": "..."}}]
}}

热榜标题列表：
{chr(10).join(hot_lines)}
""".strip()

    return [
        {"role": "system", "content": system},
        {"role": "user", "content": user},
    ]


# =========================
# 5) 重试策略（按失败原因自适应：截断->加tokens/减条目）
# =========================
def _sleep_with_backoff(attempt: int, base: float, cap: float) -> None:
    wait = min(cap, base * (1.7 ** (attempt - 1)))
    wait *= (0.85 + 0.3 * random.random())
    time.sleep(wait)


def deepseek_analyze_with_retries(items: List[Dict[str, Any]]) -> Dict[str, Any]:
    last_text: Optional[str] = None
    last_err: Optional[Exception] = None
    last_finish: Optional[str] = None

    max_tokens = int(CONFIG["MAX_TOKENS_START"])
    max_each_side = int(CONFIG["MAX_ITEMS_EACH_SIDE"])
    max_ignored = int(CONFIG["MAX_IGNORED"])

    for attempt in range(1, CONFIG["MAX_DEEPSEEK_RETRIES"] + 1):
        messages = build_messages(items, max_each_side=max_each_side, max_ignored=max_ignored)

        try:
            text, raw, finish_reason = deepseek_chat_raw(
                api_key=CONFIG["DEEPSEEK_API_KEY"],
                base_url=CONFIG["DEEPSEEK_BASE_URL"],
                model=CONFIG["DEEPSEEK_MODEL"],
                messages=messages,
                timeout=CONFIG["DEEPSEEK_TIMEOUT"],
                temperature=CONFIG["TEMPERATURE"],
                max_tokens=max_tokens,
            )
            last_text, last_finish = text, finish_reason

            # 若 finish_reason=length，官方说明可能被截断：直接走“加tokens/减条目”重试逻辑
            if finish_reason == "length":
                raise ValueError("finish_reason=length (likely truncated JSON)")

            obj = strict_parse_json(text)
            validate_result(obj)
            return obj

        except TemporaryAPIError as e:
            last_err = e
            # Retry-After 优先，否则指数退避
            if e.retry_after is not None and e.retry_after > 0:
                time.sleep(min(CONFIG["MAX_BACKOFF_SECONDS"], e.retry_after))
            else:
                _sleep_with_backoff(attempt, base=1.0, cap=CONFIG["MAX_BACKOFF_SECONDS"])

        except Exception as e:
            last_err = e

            # —— 解析失败/疑似截断：优先加 max_tokens；到上限后再减输出条数
            msg = str(e)

            # 经验判断：缺右括号/截断/finish_reason length
            likely_trunc = ("truncated" in msg.lower()) or ("finish_reason=length" in msg) or (
                last_text is not None and not clean_text(last_text).strip().endswith("}")
            )

            if likely_trunc:
                if max_tokens < CONFIG["MAX_TOKENS_MAX"]:
                    max_tokens = min(CONFIG["MAX_TOKENS_MAX"], int(max_tokens * 1.4) + 200)
                else:
                    # tokens 已到顶：减条目数，保证能完整输出
                    if max_each_side > CONFIG["MIN_ITEMS_EACH_SIDE"]:
                        max_each_side = max(CONFIG["MIN_ITEMS_EACH_SIDE"], max_each_side - 2)
                    if max_ignored > CONFIG["MIN_IGNORED"]:
                        max_ignored = max(CONFIG["MIN_IGNORED"], max_ignored - 3)
            time.sleep(CONFIG["BASE_RETRY_SLEEP_SECONDS"])

    if CONFIG.get("DEBUG_SHOW_RAW_ON_FAIL") and last_text:
        print("\n[DEBUG] Last finish_reason:", last_finish)
        print("[DEBUG] Last model output (first 1200 chars):")
        print(clean_text(last_text)[:1200])

    raise RuntimeError(
        f"Failed to obtain valid JSON after {CONFIG['MAX_DEEPSEEK_RETRIES']} attempts. "
        f"Last error: {last_err}"
    )


# =========================
# 6) DataFrame + 展示
# =========================
def to_df(lst: List[Dict[str, Any]]) -> pd.DataFrame:
    if not lst:
        return pd.DataFrame()
    df = pd.DataFrame(lst).copy()
    if "industries" in df.columns:
        df["industries"] = df["industries"].apply(lambda x: "，".join(x) if isinstance(x, list) else x)
    return df


def show_df(df: pd.DataFrame):
    if display is not None:
        display(df)
    else:
        print(df.to_string(index=False))


# =========================
# 7) 弹出窗口展示（浏览器打开本地 HTML）
# =========================
def popup_show_dfs(bullish_df: pd.DataFrame, bearish_df: pd.DataFrame, title: str, output_dir: str):
    os.makedirs(output_dir, exist_ok=True)

    def _df_to_html(df: pd.DataFrame) -> str:
        if df is None or df.empty:
            return "<p><em>（无数据）</em></p>"
        return df.to_html(index=False, escape=True)

    html = f"""
<!doctype html>
<html lang="zh">
<head>
<meta charset="utf-8">
<title>{title}</title>
<style>
body {{ font-family: -apple-system,BlinkMacSystemFont,"Segoe UI",Roboto,Arial; margin: 20px; }}
h1 {{ margin-bottom: 12px; }}
h2 {{ margin-top: 22px; }}
table {{ border-collapse: collapse; width: 100%; }}
th, td {{ border: 1px solid #ddd; padding: 8px; vertical-align: top; }}
th {{ background: #f6f6f6; }}
tr:nth-child(even) {{ background: #fafafa; }}
.small {{ color: #666; font-size: 12px; }}
</style>
</head>
<body>
<h1>{title}</h1>
<div class="small">生成时间：{datetime.now().strftime("%Y-%m-%d %H:%M:%S")}</div>

<h2>📈 可能利多（正面催化）</h2>
{_df_to_html(bullish_df)}

<h2>📉 可能利空（负面冲击）</h2>
{_df_to_html(bearish_df)}

</body>
</html>
""".strip()

    ts = datetime.now().strftime("%Y%m%d_%H%M%S")
    filepath = os.path.abspath(os.path.join(output_dir, f"popup_{ts}.html"))
    with open(filepath, "w", encoding="utf-8") as f:
        f.write(html)

    webbrowser.open(f"file://{filepath}", new=1)


# =========================
# 8) 保存 Excel（同一文件两张 sheet）
# =========================
def save_dfs_to_excel(bullish_df: pd.DataFrame, bearish_df: pd.DataFrame, ts_str: str, output_dir: str) -> str:
    os.makedirs(output_dir, exist_ok=True)

    base_name = f"hotsearch_{ts_str}.xlsx"
    path = os.path.join(output_dir, base_name)

    if os.path.exists(path):
        i = 1
        while True:
            alt = os.path.join(output_dir, f"hotsearch_{ts_str}_{i}.xlsx")
            if not os.path.exists(alt):
                path = alt
                break
            i += 1

    with pd.ExcelWriter(path, engine="openpyxl") as writer:
        (bullish_df if bullish_df is not None else pd.DataFrame()).to_excel(writer, sheet_name="利多", index=False)
        (bearish_df if bearish_df is not None else pd.DataFrame()).to_excel(writer, sheet_name="利空", index=False)

    return os.path.abspath(path)


# =========================
# 9) 运行一次
# =========================
def run_once() -> Dict[str, Any]:
    items = fetch_toutiao_hot(CONFIG["TOP_K"], CONFIG["TOUTIAO_TIMEOUT"])
    result = deepseek_analyze_with_retries(items)

    ts_display = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    ts_fname = datetime.now().strftime("%Y%m%d_%H%M%S")

    bullish_df = to_df(result.get("bullish", []))
    bearish_df = to_df(result.get("bearish", []))

    print(f"\n===== Run @ {ts_display} =====")
    print(f"Fetched {len(items)} hot topics. "
          f"Model returned {len(result.get('bullish', []))} bullish / {len(result.get('bearish', []))} bearish.\n")

    print("📈 可能利多（正面催化）")
    show_df(bullish_df)

    print("\n📉 可能利空（负面冲击）")
    show_df(bearish_df)

    if CONFIG.get("ENABLE_SAVE_EXCEL"):
        saved_path = save_dfs_to_excel(
            bullish_df=bullish_df,
            bearish_df=bearish_df,
            ts_str=ts_fname,
            output_dir=CONFIG["OUTPUT_DIR"],
        )
        print(f"\n💾 已保存 Excel：{saved_path}")

    if CONFIG.get("ENABLE_POPUP"):
        title = f"{CONFIG.get('POPUP_TITLE_PREFIX', '热搜分析')} - {ts_display}"
        popup_show_dfs(bullish_df, bearish_df, title=title, output_dir=CONFIG["OUTPUT_DIR"])

    return result


# =========================
# 10) 定时循环（可选）
# =========================
def run_scheduler():
    interval_sec = int(CONFIG["INTERVAL_MINUTES"] * 60)
    print(f"Scheduler started. Interval={CONFIG['INTERVAL_MINUTES']} minutes. Stop with Kernel Interrupt (⛔).")
    while True:
        try:
            run_once()
        except Exception as e:
            print(f"[ERROR] {e}")
        time.sleep(interval_sec)


# =========================
# 11) 执行入口
# =========================
_ = run_once()

if CONFIG["ENABLE_SCHEDULER"]:
    run_scheduler()



===== Run @ 2025-12-29 09:15:25 =====
Fetched 30 hot topics. Model returned 3 bullish / 4 bearish.

📈 可能利多（正面催化）


,rank,topic,industries,mechanism,confidence
0,3,点燃冬季文旅消费新热潮,旅游，文化娱乐,政策或活动刺激消费需求，利好相关行业,0.7
1,9,深圳一豪宅楼盘2小时销售超100亿元,房地产,豪宅热销可能提振市场信心，利好房地产板块,0.6
2,25,湘超冠军永州再迎300万捐赠,体育,捐赠可能支持体育产业发展，利好体育相关,0.5



📉 可能利空（负面冲击）


,rank,topic,industries,mechanism,confidence
0,1,东部战区军演 舰机多向抵近台岛,国防军工,地缘政治紧张可能引发市场避险情绪，利空,0.8
1,4,围岛链条越勒越紧 解放军演习区公布,国防军工,军事演习加剧不确定性，可能影响相关板块,0.7
2,5,湖北天门通报一起车辆落水事件,汽车，保险,安全事故可能引发监管关注，利空相关行业,0.4
3,26,刑释人员当上国企高管 问题出在哪,国企改革,负面新闻可能影响国企形象和投资者信心,0.5



💾 已保存 Excel：/Users/pz/VScode/Stock_Ana/hotsearch_stock_outputs/hotsearch_20251229_091525.xlsx


# 只使用qwen

In [1]:
# ==========================================
# 今日头条热榜 -> 千问分析 -> 利多/利空展示
# 稳定版：JSON-only + 本地严格校验 + 自动重试（最多N次）
# 新增：
#  1) 更稳两处小改动（提高重试次数 + 禁止字符串里出现 ASCII 双引号）
#  2) 可选“弹出窗口”（用浏览器打开本地 HTML 展示两个 DF）
#  3) 每次运行保存 Excel（同一文件两张 sheet：利多/利空；文件名=时间戳，自动避免重名）
# ==========================================

# =========================
# A) 参数区（你主要改这里）
# =========================
CONFIG = {
    # 定时运行
    "ENABLE_SCHEDULER": False,    # True=循环定时跑；False=只跑一次
    "INTERVAL_MINUTES": 30,       # 建议 10~20 也可以

    # 热搜抓取
    "TOP_K": 30,
    "TOUTIAO_TIMEOUT": 12,

    # 千问（OpenAI 兼容接口）
    "DASHSCOPE_API_KEY": "sk-b8a5d51908184663854d1b5c8e73e25e",
    "QWEN_MODEL": "qwen-plus",
    "QWEN_BASE_URL": "https://dashscope.aliyuncs.com/compatible-mode/v1",
    "QWEN_TIMEOUT": 60,

    # 稳定输出建议温度 0
    "TEMPERATURE": 0.0,
    "MAX_TOKENS": 1200,

    # 输出控制
    "MAX_ITEMS_EACH_SIDE": 12,

    # ✅ 小改动 1：提高重试次数（更稳）
    "MAX_QWEN_RETRIES": 10,
    "RETRY_SLEEP_SECONDS": 0.6,

    # 调试
    "DEBUG_SHOW_RAW_ON_FAIL": True,   # 最终失败时打印原始输出片段

    # 结果展示：弹出窗口（浏览器新窗口/新标签打开 HTML）
    "ENABLE_POPUP": True,            # True 时每次运行弹出 HTML 窗口展示两个 DF
    "POPUP_TITLE_PREFIX": "热搜利多/利空分析",

    # 结果保存：Excel
    "ENABLE_SAVE_EXCEL": True,
    "OUTPUT_DIR": "./hotsearch_stock_outputs",   # 保存目录
}

# =========================
# B) 依赖
# =========================
import os
import time
import json
import re
import webbrowser
from datetime import datetime
from typing import List, Dict, Any, Tuple, Optional

import requests
import pandas as pd

try:
    from IPython.display import display  # type: ignore
except Exception:
    display = None

# DataFrame 显示：全部行列 + 完整字符串（你要求显示全部内容）
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 0)
pd.set_option("display.max_colwidth", None)
pd.set_option("display.expand_frame_repr", False)


# =========================
# 1) 抓取今日头条热榜
# =========================
def fetch_toutiao_hot(top_k: int = 30, timeout: int = 12) -> List[Dict[str, Any]]:
    url = "https://www.toutiao.com/hot-event/hot-board/?origin=toutiao_pc"
    headers = {
        "User-Agent": (
            "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
            "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0 Safari/537.36"
        ),
        "Referer": "https://www.toutiao.com/",
        "Accept": "application/json,text/plain,*/*",
    }
    r = requests.get(url, headers=headers, timeout=timeout)
    r.raise_for_status()
    payload = r.json()

    data = payload.get("data")
    if not isinstance(data, list):
        raise ValueError(f"Unexpected response schema. Top keys: {list(payload.keys())}")

    items: List[Dict[str, Any]] = []
    for i, it in enumerate(data[:top_k], start=1):
        title = it.get("Title") or it.get("title") or ""
        hot_value = it.get("HotValue") or it.get("hot_value") or 0
        link = it.get("Url") or it.get("url") or ""

        try:
            hot_value_num = float(hot_value)
        except Exception:
            hot_value_num = None

        items.append({
            "rank": i,
            "title": str(title).strip(),
            "hot_value": hot_value_num,
            "url": str(link).strip(),
        })

    return [x for x in items if x["title"]]


# =========================
# 2) 千问调用（OpenAI 兼容 chat/completions）
# =========================
def qwen_chat_raw(
    api_key: str,
    base_url: str,
    model: str,
    messages: List[Dict[str, str]],
    timeout: int,
    temperature: float,
    max_tokens: int,
) -> Tuple[str, Dict[str, Any]]:
    """
    返回 (text, raw_json)
    """
    endpoint = base_url.rstrip("/") + "/chat/completions"
    headers = {"Authorization": f"Bearer {api_key}", "Content-Type": "application/json"}

    body: Dict[str, Any] = {
        "model": model,
        "messages": messages,
        "temperature": temperature,
        "max_tokens": max_tokens,
        # 尽量提示输出 JSON（核心仍靠重试 + prompt 约束 + 本地校验）
        "response_format": {"type": "json_object"},
    }

    r = requests.post(endpoint, headers=headers, json=body, timeout=timeout)
    if not r.ok:
        raise RuntimeError(f"HTTP {r.status_code}: {r.text[:1500]}")
    raw = r.json()

    msg = raw["choices"][0]["message"]

    # 有些实现会走 tool_calls arguments
    tool_calls = msg.get("tool_calls")
    if isinstance(tool_calls, list) and tool_calls:
        fn = tool_calls[0].get("function", {})
        args = fn.get("arguments")
        if isinstance(args, str) and args.strip():
            return args, raw

    content = msg.get("content")
    if isinstance(content, str):
        return content, raw

    return json.dumps(msg, ensure_ascii=False), raw


# =========================
# 3) JSON 清理/解析/校验
# =========================
def clean_text(s: str) -> str:
    s = s.strip()

    # 去掉 ```json``` 包裹
    s = re.sub(r"^```(?:json)?\s*", "", s, flags=re.IGNORECASE)
    s = re.sub(r"\s*```$", "", s)

    # 中文引号 -> 英文引号（避免解析失败）
    s = s.replace("“", '"').replace("”", '"').replace("‘", "'").replace("’", "'")

    # JSON 不允许 NaN/Infinity
    s = re.sub(r"\bNaN\b", "null", s)
    s = re.sub(r"\bInfinity\b", "null", s)
    s = re.sub(r"\b-Infinity\b", "null", s)

    # 尾逗号（常见）
    s = re.sub(r",\s*([}\]])", r"\1", s)

    return s


def strict_parse_json(s: str) -> Dict[str, Any]:
    """
    严格：必须是 JSON object；并且清理后必须以 { 开头、} 结尾（防止夹带文本）
    """
    s2 = clean_text(s)
    s2_strip = s2.strip()
    if not (s2_strip.startswith("{") and s2_strip.endswith("}")):
        raise ValueError("Output is not a pure JSON object (missing { } boundaries).")

    obj = json.loads(s2_strip)
    if not isinstance(obj, dict):
        raise ValueError("JSON is not an object.")
    return obj


def validate_result(obj: Dict[str, Any]) -> None:
    for k in ["bullish", "bearish", "ignored"]:
        if k not in obj:
            raise ValueError(f"Missing key: {k}")
        if not isinstance(obj[k], list):
            raise ValueError(f"Key '{k}' must be a list.")


# =========================
# 4) Prompt（更硬约束 JSON-only）
#    ✅ 小改动 2：禁止字符串里出现 ASCII 双引号 "
# =========================
def build_messages(items: List[Dict[str, Any]], max_each_side: int) -> List[Dict[str, str]]:
    hot_lines = []
    for it in items:
        hv = it["hot_value"]
        hv_str = f"{hv:.0f}" if isinstance(hv, (int, float)) and hv is not None else ""
        hot_lines.append(f'{it["rank"]}. {it["title"]} (HotValue={hv_str})')

    user = f"""
你会收到“今日头条热榜标题列表”。请判断哪些热搜可能引起行业/板块层面的短期股价波动。
将其分为：bullish（利多）、bearish（利空）、ignored（不太相关）。

极其重要的输出要求（必须遵守）：
- 只输出一段严格 JSON（不要 markdown、不要解释、不要任何前后缀文字）。
- 输出必须以字符 "{{" 开头，并以 "}}" 结尾。
- JSON 中不得出现 NaN/Infinity。
- confidence 必须是 0~1 的小数（不要百分号）。
- 每组最多 {max_each_side} 条，优先挑选“政策/监管、宏观数据、事故风险、重大交易、科技突破、能源/大宗、医疗/药品、金融风险”等更可能影响股价的主题。
- 不要编造具体股票代码；只给行业/板块。
- 【重要】所有字符串字段（topic/mechanism/reason/industries）中禁止出现 ASCII 双引号字符 (")。
  如果需要表达引用，请用中文引号（“ ”）或括号，或用单引号（'）代替。

固定 JSON Schema（字段名必须一致）：
{{
  "bullish": [{{"rank": 1, "topic": "...", "industries": ["..."], "mechanism": "...", "confidence": 0.0}}],
  "bearish":  [{{"rank": 1, "topic": "...", "industries": ["..."], "mechanism": "...", "confidence": 0.0}}],
  "ignored":  [{{"rank": 1, "topic": "...", "reason": "..."}}]
}}

热榜标题列表：
{chr(10).join(hot_lines)}
""".strip()

    return [
        {"role": "system", "content": "你是严谨的中文金融信息分析助手。必须输出严格 JSON。"},
        {"role": "user", "content": user},
    ]


# =========================
# 5) 自动重试：直到拿到可解析 JSON 或超出最大次数
# =========================
def qwen_analyze_with_retries(items: List[Dict[str, Any]]) -> Dict[str, Any]:
    messages = build_messages(items, CONFIG["MAX_ITEMS_EACH_SIDE"])

    last_text: Optional[str] = None
    last_err: Optional[Exception] = None

    for attempt in range(1, CONFIG["MAX_QWEN_RETRIES"] + 1):
        try:
            text, raw = qwen_chat_raw(
                api_key=CONFIG["DASHSCOPE_API_KEY"],
                base_url=CONFIG["QWEN_BASE_URL"],
                model=CONFIG["QWEN_MODEL"],
                messages=messages,
                timeout=CONFIG["QWEN_TIMEOUT"],
                temperature=CONFIG["TEMPERATURE"],
                max_tokens=CONFIG["MAX_TOKENS"],
            )
            last_text = text

            obj = strict_parse_json(text)
            validate_result(obj)
            return obj

        except Exception as e:
            last_err = e
            time.sleep(CONFIG["RETRY_SLEEP_SECONDS"])

    # 全失败：输出最后一次原始文本片段辅助调试
    if CONFIG.get("DEBUG_SHOW_RAW_ON_FAIL") and last_text:
        print("\n[DEBUG] Last model output (first 1200 chars):")
        print(clean_text(last_text)[:1200])

    raise RuntimeError(
        f"Failed to obtain valid JSON after {CONFIG['MAX_QWEN_RETRIES']} attempts. Last error: {last_err}"
    )


# =========================
# 6) DataFrame + 展示
# =========================
def to_df(lst: List[Dict[str, Any]]) -> pd.DataFrame:
    if not lst:
        return pd.DataFrame()
    df = pd.DataFrame(lst).copy()
    if "industries" in df.columns:
        df["industries"] = df["industries"].apply(lambda x: "，".join(x) if isinstance(x, list) else x)
    return df


def show_df(df: pd.DataFrame):
    if display is not None:
        display(df)
    else:
        print(df.to_string(index=False))


# =========================
# 7) 弹出窗口展示（浏览器打开本地 HTML）
# =========================
def popup_show_dfs(bullish_df: pd.DataFrame, bearish_df: pd.DataFrame, title: str, output_dir: str):
    os.makedirs(output_dir, exist_ok=True)

    def _df_to_html(df: pd.DataFrame) -> str:
        if df is None or df.empty:
            return "<p><em>（无数据）</em></p>"
        return df.to_html(index=False, escape=True)

    html = f"""
<!doctype html>
<html lang="zh">
<head>
<meta charset="utf-8">
<title>{title}</title>
<style>
body {{ font-family: -apple-system,BlinkMacSystemFont,"Segoe UI",Roboto,Arial; margin: 20px; }}
h1 {{ margin-bottom: 12px; }}
h2 {{ margin-top: 22px; }}
table {{ border-collapse: collapse; width: 100%; }}
th, td {{ border: 1px solid #ddd; padding: 8px; vertical-align: top; }}
th {{ background: #f6f6f6; }}
tr:nth-child(even) {{ background: #fafafa; }}
.small {{ color: #666; font-size: 12px; }}
</style>
</head>
<body>
<h1>{title}</h1>
<div class="small">生成时间：{datetime.now().strftime("%Y-%m-%d %H:%M:%S")}</div>

<h2>📈 可能利多（正面催化）</h2>
{_df_to_html(bullish_df)}

<h2>📉 可能利空（负面冲击）</h2>
{_df_to_html(bearish_df)}

</body>
</html>
""".strip()

    # 用时间戳命名，避免覆盖
    ts = datetime.now().strftime("%Y%m%d_%H%M%S")
    filepath = os.path.abspath(os.path.join(output_dir, f"popup_{ts}.html"))
    with open(filepath, "w", encoding="utf-8") as f:
        f.write(html)

    # 打开浏览器（通常会新标签/新窗口；浏览器设置可能影响行为）
    webbrowser.open(f"file://{filepath}", new=1)


# =========================
# 8) 保存 Excel（同一文件两张 sheet：利多/利空；文件名=时间戳，自动避免重名）
# =========================
def save_dfs_to_excel(bullish_df: pd.DataFrame, bearish_df: pd.DataFrame, ts_str: str, output_dir: str) -> str:
    os.makedirs(output_dir, exist_ok=True)

    base_name = f"hotsearch_{ts_str}.xlsx"
    path = os.path.join(output_dir, base_name)

    # 避免重名：如果存在则追加 _1, _2 ...
    if os.path.exists(path):
        i = 1
        while True:
            alt = os.path.join(output_dir, f"hotsearch_{ts_str}_{i}.xlsx")
            if not os.path.exists(alt):
                path = alt
                break
            i += 1

    with pd.ExcelWriter(path, engine="openpyxl") as writer:
        # sheet 名按你的要求：利多/利空
        (bullish_df if bullish_df is not None else pd.DataFrame()).to_excel(writer, sheet_name="利多", index=False)
        (bearish_df if bearish_df is not None else pd.DataFrame()).to_excel(writer, sheet_name="利空", index=False)

    return os.path.abspath(path)


# =========================
# 9) 运行一次
# =========================
def run_once() -> Dict[str, Any]:
    items = fetch_toutiao_hot(CONFIG["TOP_K"], CONFIG["TOUTIAO_TIMEOUT"])
    result = qwen_analyze_with_retries(items)

    ts_display = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    ts_fname = datetime.now().strftime("%Y%m%d_%H%M%S")

    bullish_df = to_df(result.get("bullish", []))
    bearish_df = to_df(result.get("bearish", []))

    print(f"\n===== Run @ {ts_display} =====")
    print(f"Fetched {len(items)} hot topics. "
          f"Model returned {len(result.get('bullish', []))} bullish / {len(result.get('bearish', []))} bearish.\n")

    print("📈 可能利多（正面催化）")
    show_df(bullish_df)

    print("\n📉 可能利空（负面冲击）")
    show_df(bearish_df)

    # 保存 Excel
    if CONFIG.get("ENABLE_SAVE_EXCEL"):
        saved_path = save_dfs_to_excel(
            bullish_df=bullish_df,
            bearish_df=bearish_df,
            ts_str=ts_fname,
            output_dir=CONFIG["OUTPUT_DIR"],
        )
        print(f"\n💾 已保存 Excel：{saved_path}")

    # 弹出窗口（浏览器）
    if CONFIG.get("ENABLE_POPUP"):
        title = f"{CONFIG.get('POPUP_TITLE_PREFIX', '热搜分析')} - {ts_display}"
        popup_show_dfs(bullish_df, bearish_df, title=title, output_dir=CONFIG["OUTPUT_DIR"])

    return result


# =========================
# 10) 定时循环（可选）
# =========================
def run_scheduler():
    interval_sec = int(CONFIG["INTERVAL_MINUTES"] * 60)
    print(f"Scheduler started. Interval={CONFIG['INTERVAL_MINUTES']} minutes. Stop with Kernel Interrupt (⛔).")
    while True:
        try:
            run_once()
        except Exception as e:
            print(f"[ERROR] {e}")
        time.sleep(interval_sec)


# =========================
# 11) 执行入口
# =========================
_ = run_once()

if CONFIG["ENABLE_SCHEDULER"]:
    run_scheduler()



===== Run @ 2025-12-29 08:53:27 =====
Fetched 30 hot topics. Model returned 2 bullish / 4 bearish.

📈 可能利多（正面催化）


,rank,topic,industries,mechanism,confidence
0,7,深圳一豪宅楼盘2小时销售超100亿元,房地产，高端住宅，建筑材料,一线城市高端楼市销售回暖，可能预示地产政策放松效果显现，带动市场对房地产板块信心修复，尤其利好头部房企及建材供应链,0.85
1,3,点燃冬季文旅消费新热潮,旅游，酒店餐饮，文化娱乐,政策或活动推动冬季文旅消费，短期内提升出行及服务类消费需求，利好景区、OTA平台及连锁餐饮企业,0.75



📉 可能利空（负面冲击）


,rank,topic,industries,mechanism,confidence
0,1,东部战区军演 舰机多向抵近台岛,半导体，电子制造，航运物流,地缘政治紧张升级可能导致台海供应链扰动预期，市场担忧两岸交流中断影响芯片等产业上下游协作，引发相关科技与运输板块短期避险情绪,0.90
1,4,围岛演习！东部战区发布演训示意图,半导体，电子制造，航运物流,进一步强化军事行动可视化，加剧市场对台海局势失控的担忧，放大全球供应链中断风险预期，压制出口导向型科技与航运板块估值,0.88
2,24,大范围雨雪来了 元旦大降温趋势确定,农业，交通运输，零售,极端天气将影响农产品运输与供应，推高生鲜损耗率，同时抑制节前消费出行，对农业供应链和线下零售形成短期压力,0.70
3,29,冷空气伴我们跨年,农业，交通运输,持续低温天气延长物流中断时间，增加冷链与配送成本，进一步冲击生鲜供应稳定性，拖累农业与运输板块表现,0.65



💾 已保存 Excel：/Users/pz/VScode/Stock_Ana/hotsearch_stock_outputs/hotsearch_20251229_085327.xlsx
